In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import matplotlib.pyplot as plt
import npc_sessions

In [ ]:
session = npc_sessions.Session(
    R'\\allen\programs\mindscope\workgroups\dynamicrouting\PilotEphys\Task 2 pilot\DRpilot_662892_20230824'
    )

# Sync

In [ ]:
display(tuple((device.name, device.sampling_rate, device.start_time) for device in npc_sessions.get_ephys_timing_on_sync(session.sync_path, session.ephys_recording_dirs)))

In [ ]:
session.sync_data.meta_data

In [ ]:
fig, axes = session.sync_data.plot_diode_measured_sync_square_flips()
stim_display_times = npc_sessions.get_stim_frame_times(*session.stim_paths, sync=session.sync_data)
names = tuple(k for k, v in stim_display_times.items() if v is not None)
for idx, ax in enumerate(axes):
    ax.set_title(names[idx].stem.split('_')[0])
fig.set_size_inches(20, 10)

In [ ]:
fig, axes = session.sync_data.plot_stim_onsets()
names = tuple(k for k, v in stim_display_times.items() if v is not None)
for idx, ax in enumerate(axes):
    ax.set_title(names[idx].stem.split('_')[0])
fig.set_size_inches(20, 5 * len(axes))
fig.subplots_adjust(hspace=0.3)


# MVR

In [ ]:
import IPython.display

for info in session.video_info_data.values():
    print(info['CameraLabel']), display(IPython.display.JSON(
        {
            k:v
            for k,v in info.items() 
            if k in ('FPS', 'Duration', 'FramesRecorded', 'FramesLostCount')
        } | {'Lost': f"{100 * info['FramesLostCount'] / info['FramesRecorded']:.5f}%"}
    )
)

In [ ]:
from typing import Iterable
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cv2
import numpy as np
import datetime
import io

import upath

def camera_frame_grabs_simple(
    paths: Iterable[upath.UPath],
    num_frames_to_grab: int = 5,
) -> None:
    """Just plots evenly spaced frames, no concept of epochs.
    
    video frames across cameras aren't synced .
    """
    fig = plt.figure(figsize=[20, 3 * len(paths)], constrained_layout=True, facecolor='0.5')
    gs = gridspec.GridSpec(len(paths), num_frames_to_grab, figure=fig)
    gs.update(wspace=0.0, hspace=0.0)
    for idx, video_path in enumerate(paths):
        # get frames to plot
        v = cv2.VideoCapture(video_path.as_posix()) # TODO open with upath from cloud
        
        frame_delta = np.ceil(v.get(7) / num_frames_to_grab + 1)
        frames_of_interest = np.arange(v.get(5), v.get(7), frame_delta)

        for i, f in enumerate(frames_of_interest):
            v.set(cv2.CAP_PROP_POS_FRAMES, int(f))
            ret, frame = v.read()
            ax = fig.add_subplot(gs[idx, i])
            ax.imshow(frame)
            # ax.axis('off')
            ax.tick_params(
                top=False,
                bottom=False,
                left=False,
                right=False,
                labelleft=False,
                labelbottom=False,
            )
            ax.set_title(datetime.timedelta(seconds=f/v.get(5)), fontsize=10)
    return fig

fig = camera_frame_grabs_simple(session.video_paths)

## Verify frame times can be found on sync

In [ ]:
npc_sessions.get_video_frame_times(session.sync_data, *session.video_paths)